# Data Cleaning and Preprocessing

This notebook covers essential data cleaning and preprocessing techniques using pandas.
We'll work with a messy customer dataset and clean it step by step.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Set random seed
np.random.seed(42)

In [ ]:
# Generate sample data
n_samples = 1000

# Create customer data with various issues
data = {
    "customer_id": range(1, n_samples + 1),
    "name": [f"Customer_{i}" for i in range(1, n_samples + 1)],
    "age": np.random.randint(18, 90, n_samples),
    "email": [
        f"customer_{i}@email.com" if i % 10 != 0 else None
        for i in range(1, n_samples + 1)
    ],
    "purchase_amount": np.random.normal(500, 150, n_samples),
    "purchase_date": pd.date_range(start="2023-01-01", periods=n_samples),
    "category": np.random.choice(
        ["Electronics", "Clothing", "Books", "Home", None], n_samples
    ),
    "satisfaction_score": np.random.choice([1, 2, 3, 4, 5, None], n_samples),
    "phone_number": [
        f"+1-555-{str(i).zfill(4)}" if i % 15 != 0 else "invalid"
        for i in range(1, n_samples + 1)
    ],
}

# Create DataFrame
df = pd.DataFrame(data)

In [ ]:
df.head()

In [ ]:
# Introduce some messiness
df.loc[np.random.choice(df.index, 50), "age"] = None  # Missing ages
df.loc[np.random.choice(df.index, 30), "purchase_amount"] = (
    np.nan
)  # Missing purchase amounts
df.loc[np.random.choice(df.index, 20), "age"] = -1  # Invalid ages
df.loc[np.random.choice(df.index, 20), "purchase_amount"] = (
    -100
)  # Invalid purchase amounts

In [ ]:
df.info()

# 1. Initial Data Exploration

In [ ]:
print("\nMissing values count:")
print(df.isnull().sum())

In [ ]:
print("\nBasic statistics:")
print(df.describe())

# 2. Handling Missing Values

In [ ]:
# Check missing value percentages
missing_percentages = (df.isnull().sum() / len(df)) * 100
print("\nMissing value percentages:")
print(missing_percentages)

In [ ]:
# Fill missing emails with 'unknown'
df["email"] = df["email"].fillna("unknown")

In [ ]:
# Fill missing categories with 'Other'
df["category"] = df["category"].fillna("Other")

> **Median** The median is the middle value when data is ordered. In pandas, you can calculate it using .median().

In [ ]:
# Fill missing satisfaction scores with median
df["satisfaction_score"] = df["satisfaction_score"].fillna(
    df["satisfaction_score"].median()
)

In [ ]:
# Fill missing ages with median
df["age"] = df["age"].fillna(df["age"].median())

In [ ]:
# Fill missing purchase amounts with mean
df["purchase_amount"] = df["purchase_amount"].fillna(df["purchase_amount"].mean())

In [ ]:
print("\nRemaining missing values after handling:")
print(df.isnull().sum())

# 3. Handling Invalid Values

In [ ]:
# Fix invalid ages
df["age"] = df["age"].apply(lambda x: None if x < 0 else x)
df["age"] = df["age"].fillna(df["age"].median())

In [ ]:
# Fix invalid purchase amounts
df["purchase_amount"] = df["purchase_amount"].apply(lambda x: None if x < 0 else x)
df["purchase_amount"] = df["purchase_amount"].fillna(df["purchase_amount"].mean())

# 4. Data Validation and Constraints

In [ ]:
# Ensure age is within reasonable range
df["age"] = df["age"].clip(lower=18, upper=90)

In [ ]:
# Round purchase amounts to 2 decimal places
df["purchase_amount"] = df["purchase_amount"].round(2)

In [ ]:
# Ensure satisfaction scores are integers
df["satisfaction_score"] = df["satisfaction_score"].round().astype(int)

# 5. Feature Engineering

In [ ]:
# Extract month from purchase date
df["purchase_month"] = df["purchase_date"].dt.month

In [ ]:
# Create purchase amount categories
df["purchase_category"] = pd.qcut(
    df["purchase_amount"], q=4, labels=["Low", "Medium", "High", "Premium"]
)

In [ ]:
# Create age groups
df["age_group"] = pd.cut(
    df["age"],
    bins=[17, 25, 35, 50, 65, 90],
    labels=["18-25", "26-35", "36-50", "51-65", "65+"],
)

In [ ]:
print("\nNew features preview:")
print(df[["purchase_month", "purchase_category", "age_group"]].head())

# 6. Data Visualization

In [ ]:
# Create visualizations
plt.figure(figsize=(15, 10))

In [ ]:
# Plot 1: Age distribution
plt.subplot(2, 2, 1)
sns.histplot(data=df, x="age", bins=20)
plt.title("Age Distribution")

In [ ]:
# Plot 2: Purchase amount distribution
plt.subplot(2, 2, 2)
sns.boxplot(data=df, x="category", y="purchase_amount")
plt.xticks(rotation=45)
plt.title("Purchase Amount by Category")

In [ ]:
# Plot 3: Satisfaction score distribution
plt.subplot(2, 2, 3)
sns.countplot(data=df, x="satisfaction_score")
plt.title("Satisfaction Score Distribution")

In [ ]:
# Plot 4: Purchase amount by age group
plt.subplot(2, 2, 4)
sns.boxplot(data=df, x="age_group", y="purchase_amount")
plt.xticks(rotation=45)
plt.title("Purchase Amount by Age Group")

plt.tight_layout()
plt.show()

# 7. Final Data Quality Check

In [ ]:
print("\nFinal dataset info:")
print(df.info())

In [ ]:
print("\nFinal dataset summary statistics:")
print(df.describe())

# 8. Export Clean Dataset

In [ ]:
# Export to CSV
df.to_csv("clean_customer_data.csv", index=False)
print("\nClean dataset exported to 'clean_customer_data.csv'")

# 9. Summary of Cleaning Operations

Cleaning operations performed:
1. Handled missing values for all columns
2. Fixed invalid ages and purchase amounts
3. Standardized phone numbers
4. Applied data constraints (age range, purchase amount decimals)
5. Created new features (purchase_month, purchase_category, age_group)
6. Performed data validation
7. Generated visualizations for data quality check
8. Exported clean dataset

# 10. Data Quality Metrics

In [ ]:
quality_metrics = {
    "Total Records": len(df),
    "Missing Values": df.isnull().sum().sum(),
    "Duplicate Records": len(df[df.duplicated()]),
    "Age Range": f"{df['age'].min()} - {df['age'].max()}",
    "Purchase Amount Range": f"${df['purchase_amount'].min():.2f} - ${df['purchase_amount'].max():.2f}",
    "Unique Categories": len(df["category"].unique()),
    "Average Satisfaction": df["satisfaction_score"].mean(),
}

for metric, value in quality_metrics.items():
    print(f"{metric}: {value}")